<a href="https://colab.research.google.com/github/Ash2809/English-Hindi-Translator/blob/main/English_Hindi_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hindienglish-corpora:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F200079%2F441417%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240823%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240823T131948Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1b19d0ae6dff4616ec8db799b4652381a5618b2c40dea6f615e4f0db60a0c68bbeb505b2abf653bad3670946c22f2b0e4a6a48e647ed0bc59058ad0f0ded5f6e672a07bda876e616c7bdae7a7e11c7763061bd5f594f19dd60407a1d18d7e460c0af725e371f2fbc5f4c25dd3de11713aa3a7d0f38708727566cd9fc9cf454aa893f939597b8dae4273428765de6202ad93ce8eb08fbd8ec4b5d3710e687fdf88431189522d65cd1fea7c0ed6f61f90b65e02a475eedc51122585a04bc0895796a5b743bcbb09924b84524264c8f829a2ec35ab7c3a47556680cb24dace177385af62af8ffe2cc9fb18f74cf773023dbc596c0d6d5d2fd5166a9ec00915caaa0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 14623176 bytes downloaded
Downloaded and uncompressed: hindienglish-corpora
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Embedding, Input, Masking
from keras.models import load_model, save_model, Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from wordcloud import STOPWORDS, WordCloud
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
dataset = pd.read_csv(r"/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")
dataset.head(5)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [6]:
dataset.drop(columns = ["source"], axis = 1, inplace = True)

In [7]:
print(dataset.isnull().sum())
dataset.dropna(inplace = True)

english_sentence    2
hindi_sentence      0
dtype: int64


In [8]:
print(dataset.duplicated().sum())
dataset.drop_duplicates(inplace = True)
dataset.shape

2780


(124825, 2)

In [11]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    stopwords_set = set(stopwords.words("english"))
    lematizer = WordNetLemmatizer()

    text = " ".join(lematizer.lemmatize(word) for word in text.split() if word not in stopwords_set)
    return text

In [12]:
dataset["english_sentence"] = dataset["english_sentence"].apply(preprocess)
dataset.head(4)

,english_sentence,hindi_sentence
0,politician permission need done,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,id like tell one child,मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,percentage even greater percentage india,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,really mean theyre bad paying attention,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते


In [13]:
def add_tokens(text):
    START_TOKEN = "<start>"
    END_TOKEN = "<end>"

    return START_TOKEN + text + END_TOKEN


In [14]:
dataset["hindi_sentence"] = dataset["hindi_sentence"].apply(add_tokens)
dataset.head(4)

,english_sentence,hindi_sentence
0,politician permission need done,<start>राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,id like tell one child,<start>मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,percentage even greater percentage india,<start>यह प्रतिशत भारत में हिन्दुओं प्रतिशत से...
3,really mean theyre bad paying attention,<start>हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...


In [15]:
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(dataset["english_sentence"])
english_sequences = english_tokenizer.texts_to_sequences(dataset["english_sentence"])

In [16]:
hindi_tokenizer = Tokenizer()
hindi_tokenizer.fit_on_texts(dataset["hindi_sentence"])
hindi_sequences = hindi_tokenizer.texts_to_sequences(dataset["hindi_sentence"])

In [17]:
max_len_eng = max(len(seq) for seq in english_sequences)
max_len_hindi = max(len(seq) for seq in hindi_sequences)
max_len_eng, max_len_hindi

(219, 419)

In [18]:
english_padded = pad_sequences(english_sequences, maxlen= max_len_eng, padding = "post")
hindi_padded = pad_sequences(hindi_sequences, maxlen= max_len_hindi, padding = "post")

In [19]:
vocab_size = len(english_tokenizer.word_index)+1
vocab_size

70812

In [20]:
encoder_input = Input(shape = (None,))
encoder_embeddings = Embedding(input_dim = vocab_size, output_dim = 256)(encoder_input)
encoder_masking = Masking(mask_value=0.0)(encoder_embeddings)
encoder_LSTM = LSTM(units = 256, return_state = True)
_, state_h, state_c = encoder_LSTM(encoder_masking)

decoder_input = Input(shape = (None,))
decoder_embeddings = Embedding(input_dim = len(hindi_tokenizer.word_index)+1, output_dim = 256)(decoder_input)
decoder_masking = Masking(mask_value = 0.0)(decoder_embeddings)
decoder_LSTM = LSTM(units = 256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_LSTM(decoder_masking, initial_state = [state_h, state_c])

decoder_Dense = Dense(units = len(hindi_tokenizer.word_index)+1, activation = "softmax")
decoder_output = decoder_Dense(decoder_outputs)

model = Model(inputs = [encoder_input, decoder_input], outputs = decoder_output)

In [21]:
model.compile(optimizer = "Adam", metrics = ["Accuracy"], loss = "sparse_categorical_crossentropy")

In [22]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 256)      │     18,127,872 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None, 256)      │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 256)      │     19,719,680 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking (Masking)         │ (None, None, 256)      │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ any (Any)                 │ (None, None)           │              0 │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking_1 (Masking)       │ (None, None, 256)      │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        525,312 │ masking[0][0],         │
│                           │ 256), (None, 256)]     │                │ any[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │        525,312 │ masking_1[0][0],       │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 77030)    │     19,796,710 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 58,694,886 (223.90 MB)

 Trainable params: 58,694,886 (223.90 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
callback = EarlyStopping(monitor = "val_loss", patience = 3, restore_best_weights = True)
h = model.fit([english_padded, hindi_padded[:, :-1]], hindi_padded[:, :-1], batch_size = 13, callbacks = [callback], epochs = 3, validation_split = 0.2)

Epoch 1/3
7682/7682 ━━━━━━━━━━━━━━━━━━━━ 4523s 588ms/step - Accuracy: 0.9723 - loss: 0.3628 - val_Accuracy: 0.9954 - val_loss: 0.0447
Epoch 2/3
3460/7682 ━━━━━━━━━━━━━━━━━━━━ 36:39 521ms/step - Accuracy: 0.9960 - loss: 0.0370

KeyboardInterrupt: 

In [24]:
model.save("translator.h5")